# Projet 1 - Modélisation du slicing dans les réseaux 5G

**1)** Modiliser le nombre moyen de clients dans une file M/M/S/S à l’état stationnaire sans calculer de factorielle.

En reprennant relation de récurrence de la formule de d’Erlang-B : 

$$
\frac{1}{\mathrm{Erl}_B[\rho, 0]} = 1 \\
\frac{1}{\mathrm{Erl}_B[\rho, S]} = 1 + \frac{S}{\rho\, \mathrm{Erl}_B[\rho, S-1]}.
$$

Pour rappel, la file M/M/S/S est definis comme :
- Arrivées de parametre $\lambda$
- Service de parametre $\mu$
- S serveur, pas de buffer
- $\rho$ = $\frac{\lambda}{\mu}$
- Modele a perte, car pas de file d'attente, si les S serveurs sont occupé, le client est perdu.

Je pose N, le nombre de clients a l'etat stationnaire (ne varie pas), alors comme vue dans le cours, le nombre moyen de clients est donné par  :

$$ \mathbb{E}[N] = \sum_{k=1}^{S} k \cdot \pi_s(k) $$

Or le $\pi_s(i)$ est la proba stationnaire d'avoir i clients dans le systeme donc : 
$$\pi_s(i) = \frac{\dfrac{\rho^i}{i!}}{\sum_{j=0}^{S} \dfrac{\rho^j}{j!}} = \frac{\rho^i}{i!}\pi_s(0)$$ 

Donc on trouve cette fomule finale : $$ \mathbb{E}[N] = \sum_{n=1}^{S} n \cdot \frac{\dfrac{\rho^n}{n!}}{\sum_{j=0}^{S} \dfrac{\rho^j}{j!}}$$

In [1]:
def mean_number_waiting_customers(arrival_rate,service_rate,nb_of_servers):
    rho = arrival_rate / service_rate #(Fomule de \frac{\lambda}{\mu})
    value = 1.0  
    pi_0 = value
    numerateur = 0.0

    #Fais la reccurance sur tout les thermes et traite le numerateur et deniminateur separement pour ne pas faire de factorielle
    for n in range(1, nb_of_servers + 1):
        value *= rho / n  
        pi_0 += value
        numerateur += n * value  
    return numerateur / pi_0


Via les résultats connus sur le modèle **M/M/∞**, on sait que lorsque $ S \rightarrow +\infty $ :

$$
\pi_{\infty}(i) = \frac{\dfrac{\rho^i}{i!}}{\sum_{j=0}^{\infty} \dfrac{\rho^j}{j!}} = e^{-\rho} \frac{\rho^i}{i!}
$$

Ceci correspond exactement à une **loi de Poisson** de paramètre $ \rho $.

Par definition d'une loi de poission, le **nombre moyen de clients dans le système** quand $ S \rightarrow +\infty $ est :

$$
\mathbb{E}[N] = \sum_{i=0}^{\infty} i \cdot \pi_{\infty}(i) = \rho
$$

Cela montre que, dans le cas d'une infinité de serveurs, le système ne sature jamais et le **nombre moyen de clients** est simplement donné par **le rapport entre le taux d’arrivée et le taux de service**.

Pour ce qui est de $Erl_B[ρ,S]$ << 1, cela veux dire peut de perte, donc pas d'attente, c'est comme si on avais des serveurs infini du point de vue du client client. 


### 3-Modélisation

Dans cette partie, nous avons 2 types de clients : 

Type 1 (priorité plus elevé):
- Arrive selon $PP(\lambda_1)$
- Durée de service $Exp(\mu_1)$
- Non bufferisés
- Capacité de préemption 
<br>

Type 2 :
- Arrive selon $PP(\lambda_1)$
- Durée de service $Exp(\mu_2)$
- Bufferisés

On note : 
- $Q_1$ = #clients de type 1 dans le systeme, respectivement $Q_2$.
- $S_1$ = #nombre de serveurs occupés par des clients de type 1, respectivement $S_2$.
- B = #clients de type 2 dans le buffer


##### 3.1)
Si j'ecris mathematiquement les definitions : 

- $S_1 + S_2 < S$ $\Leftrightarrow$ Il reste des serveurs libres, donc un client de type 1 peut être servi immédiatement.

- $S_1 + S_2 = S$ $\Leftrightarrow$ Tous les serveurs sont occupés. Grâce à la préemption, un client de type 2 est déplacé dans le buffer pour libérer un serveur pour un client de type 1.

- $S_1 = S$ $\Leftrightarrow$ Tous les serveurs sont occupés par des clients de type 1. Un nouveau client de type 1 est alors perdu.

- $ Q_1 = S_1 $, car les clients de type 1 ne sont jamais bufferisés.

- $ Q_2 = S_2 + B$, car les clients de type 2 peuvent être en service et/ou en file d’attente.

Donc

Si $ q_1 + s_2 < S $ $\Rightarrow$ il reste au moins un serveur libre.

Or, si b > 0 $\Rightarrow$ y a des clients de type 2 dans le buffer, et qu’il reste un serveur libre, le client dans le buffer serait automatiquement mis dans un serveur.

Donc la situation $ b > 0 $ n’est valable qu’avec un système saturé : $ q_1 + s_2 = S $.  
Donc j'en conclu que :

$$
q_1 + s_2 < S \quad \Rightarrow \quad b = 0
$$


#### 3.3)

En reprenant les définitions du dessus, la transitions du processus $ (Q_1, Q_2) $ sont :

- $ (q_1, q_2) \rightarrow (q_1 + 1, q_2) = \lambda_1 $

- $ (q_1, q_2) \rightarrow (q_1, q_2 + 1) = \lambda_2 $

- $ (q_1, q_2) \rightarrow (q_1 - 1, q_2) = \min(q_1, S) \cdot \mu_1 $  
  (car on peut servir au plus $ S $ clients de type 1, grâce à la préemption)

- $ (q_1, q_2) \rightarrow (q_1, q_2 - 1) = \min(q_2, S - q_1) \cdot \mu_2 $  
  (car les clients de type 1 sont prioritaires, donc on peut servir au plus $ S - q_1 $ clients de type 2 si il y a $q_1$ clients de type 1 dans le systeme)




### 3.4)


In [ ]:
import numpy as np

def erlang_b(rho, S):
    value = 1.0
    denominator = value
    for n in range(1, S + 1):
        value *= rho / n
        denominator += value
    return value / denominator

# Parametre pour le systeme
S = 10
mu1 = 2
mu2 = 1
lambda1 = 4
lambda2 = 3
rho1 = lambda1 / mu1
rho2 = lambda2 / mu2


temps_simulation = 100000
t = 0
Q1 = 0
Q2 = 0
time_in_Q1_full = 0

np.random.seed(0)

while t < temps_simulation:
    evenements = []
    
    if Q1 < S:
        evenements.append(('arriver_T1', lambda1)) #Formule de transition (q1,q2) -> (q1+1,q2)
    else:
        evenements.append(('arrive_T1_perdu', 0))  #Client type 1 perdu (Q1=S)
    
    evenements.append(('depart_T1', min(Q1, S) * mu1)) #Formule de transition (q1,q2) -> (q1-1,q2)
    evenements.append(('arriver_T2', lambda2)) #Formule de transition (q1,q2) -> (q1,q2+1)
    evenements.append(('depart_T2', min(Q2, S - Q1) * mu2)) #Formule de transition (q1,q2) -> (q1,q2−1)
    print(evenements)
    
    events, taux = zip(*evenements)
    total = sum(taux)

    if total == 0:
        break

    dt = np.random.exponential(1 / total)
    t += dt

    if Q1 == S:
        time_in_Q1_full += dt

    prochain_event = np.random.choice(events, p=np.array(taux) / total)

    if prochain_event == 'arriver_T1' and Q1 < S:
        Q1 += 1
    elif prochain_event == 'depart_T1' and Q1 > 0:
        Q1 -= 1
    elif prochain_event == 'arriver_T2':
        Q2 += 1
    elif prochain_event == 'depart_T2' and Q2 > 0 and min(Q2, S - Q1) > 0:
        Q2 -= 1


empirical_prob = time_in_Q1_full / temps_simulation
theoretical_prob = erlang_b(rho1, S)

print(f"Proportion empirique de Q1 = S : {empirical_prob:.5f}")
print(f"Probabilité théorique Erlang-B : {theoretical_prob:.5f}")


Proportion empirique de Q1 = S : 0.00003
Probabilité théorique Erlang-B : 0.00004
